# DeepLIIF Inference Request Demo
This notebook is inteded to help show the steps of how to deploy a model using cli_mlops.py and test that the model is working.

Model gets staged and deployed using the commands in this notebook.  There are examples on how to test an inference request
with an image from the storage volume and with a serialized image in the request.

In [ ]:
import wml_sdk_utils as wml_util
import os
import wmla_edi_utils as edi
from PIL import Image
import requests

os.environ['SPACE_ID'] = "81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7"
os.environ['REST_SERVER'] = "https://wmla-console-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/"

DLIM_PATH = "/userfs"
# Add to path
if not DLIM_PATH in os.environ['PATH']:
    os.environ['PATH'] = os.environ['PATH'] + f':{DLIM_PATH}'
    
DEPLOYMENT_URL = "https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/deepliif-test"

os.chdir('/userfs')

## Stage and deploy model
Model needs to be staged to the WML assets space and then deployed to WMLA.  The following command needs to be run in a terminal session

In [ ]:

!python cli_mlops.py prepare stage --path-yml deployment_metadata_cli_example.yml --path-model /mnts/AdditionalDeepLIIFVolume/deepliif-ws-output/Test_Model_wendy_ws_serialized3 --path-dependency /userfs/wmla-deployment/edi-deployment-dirs/deepliif-base -f

In [ ]:
!python cli_mlops.py deploy stop --name deepliif-test

In [ ]:
!python cli_mlops.py deploy delete --name deepliif-test

In [ ]:
!python cli_mlops.py deploy create --model-asset-id 8ec4ed52-a390-4fa1-9114-a48ffd437bc8  --cpd-username *** --cpd-api-key *** --kernel-filename kernel.py --name deepliif-test --dlim-path /home/wsuser/bin --custom-arg CPD_USERNAME=*** --custom-arg CPD_API_KEY=*** --custom-arg VOLUME_DISPLAY_NAME=AdditionalDeepLIIFVolume

## Image on volume (recommended for large images)

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {
        'img_path_on_pvc':'edi_deployments/sample_input_images/22_2_real_A.png',
        'save_path_on_pvc':'edi_inference/output_images',
        'tile_size': 512,
        'images_to_return': 'seg_masks',
        }

In [ ]:
r = requests.post(DEPLOYMENT_URL, headers=headers,
                  json = data, verify = False)
r.status_code

In [ ]:
r.json()

## Send and return serialized image in request

In [ ]:
local_image = Image.open("/mnts/DeepLIIFData/edi_deployments/sample_input_images/22_2_real_A.png")

In [ ]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {
        'local_input_image': edi.serialize_image(local_image),
        'tile_size': 512,
        'images_to_return': 'all',
        }

In [ ]:
r2 = requests.post(DEPLOYMENT_URL, headers=headers,
                  json = data, verify = False)
r2.status_code

In [ ]:
output = r2.json()

In [ ]:
output['images'].keys()

In [ ]:
output['msg']

In [ ]:
Image.open(edi.deserialize_image(output['images']['local_SegOverlaid.png']))